# Ollama + OpenAI + Python

## 1. Určete název modelu

Pokud jste použili jiný model než "phi3:mini", změňte hodnotu v buňce níže. Tato proměnná bude použita v kódu v celém notebooku.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Nastavení klienta Open AI

Klient OpenAI se obvykle používá s OpenAI.com nebo Azure OpenAI k interakci s velkými jazykovými modely.  
Nicméně jej lze také použít s Ollama, protože Ollama poskytuje kompatibilní rozhraní OpenAI na "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Vytvoření odpovědi v chatu

Nyní můžeme použít OpenAI SDK k vytvoření odpovědi na konverzaci. Tento požadavek by měl vytvořit haiku o kočkách:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Návrh promptů

První zpráva zaslaná jazykovému modelu se nazývá "systémová zpráva" nebo "systémový prompt" a nastavuje celkové instrukce pro model.  
Můžete poskytnout vlastní upravený systémový prompt, který povede jazykový model k vytváření výstupů jiným způsobem.  
Upravte `SYSTEM_MESSAGE` níže tak, aby odpovídal jako vaše oblíbená postava z filmu nebo seriálu, nebo se inspirujte jinými systémovými prompty z [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Jakmile upravíte systémovou zprávu, zadejte první uživatelskou otázku do `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Příklady s několika ukázkami

Dalším způsobem, jak vést jazykový model, je poskytnout "několik ukázek", tedy sekvenci příkladů otázek a odpovědí, které ukazují, jak by měl model reagovat.

Níže uvedený příklad se snaží přimět jazykový model, aby se choval jako asistent výuky, tím že poskytne několik příkladů otázek a odpovědí, které by mohl dát asistent, a poté modelu položí otázku, kterou by mohl položit student.

Vyzkoušejte to nejdříve, a poté upravte `SYSTEM_MESSAGE`, `EXAMPLES` a `USER_MESSAGE` pro nový scénář.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Generování s podporou vyhledávání

RAG (Retrieval Augmented Generation) je technika, která umožňuje jazykovému modelu přesně odpovídat na otázky v konkrétní oblasti tím, že nejprve vyhledá relevantní informace ze zdroje znalostí a poté na základě těchto informací vytvoří odpověď.

Poskytli jsme místní CSV soubor s daty o hybridních autech. Níže uvedený kód načte CSV soubor, vyhledá shody s otázkou uživatele a poté vytvoří odpověď na základě nalezených informací. Všimněte si, že tento proces bude trvat déle než u předchozích příkladů, protože modelu posílá více dat. Pokud si všimnete, že odpověď stále není založena na datech, můžete zkusit systémové úpravy nebo jiné modely. Obecně je RAG efektivnější buď s většími modely, nebo s upravenými verzemi SLM.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). I když se snažíme o přesnost, mějte prosím na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádné nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
